In [1]:
import locale
import glob
import os.path
import requests
import tarfile
import sys
import codecs
from smart_open import smart_open
import re
import nltk
import numpy as np
import pickle
import itertools
from collections import Counter, OrderedDict
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from keras.preprocessing.sequence import pad_sequences
import re
from os import listdir
import string
from nltk.corpus import stopwords
from util.util_functions import getWordIdx, load_embedding_matrix_gensim
import gensim
import pickle
import keras
import tensorflow as tf
from keras.layers import Conv1D,Embedding,MaxPooling1D,Input
from keras.models import Model
import pandas as pd
from bs4 import BeautifulSoup
import string
from sklearn.model_selection import train_test_split
import pickle
import keras
from keras.models import Sequential, Model
from keras.layers.core import Dense, Dropout, Activation, Flatten, Lambda
from keras.layers.embeddings import Embedding
from keras.layers import Convolution1D, MaxPooling1D, GlobalMaxPooling1D, Input, Dense, Reshape, LSTM, GRU, Bidirectional, TimeDistributed
from keras.layers.merge import concatenate, add
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from keras import initializers
from keras.engine.topology import Layer
from keras import backend as K
import tensorflow as tf
import numpy as np
from util.util_functions import getWordIdx
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report, roc_auc_score
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score

Using TensorFlow backend.
C:\Users\wyl19\Anaconda3\envs\tensorflow_cpu\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
#define all of the functions
punctuation_list = list(string.punctuation)

# Convert text to lower-case and strip punctuation/symbols from words
def normalize_text(text):
#     norm_text = text.lower()
    # Replace breaks with spaces
    norm_text = text.replace('<br />', ' ')
    # Pad punctuation with spaces on both sides
    norm_text = re.sub(r"([\[\].\",()!?;:/])", " \\1 ", norm_text)
    return norm_text

def sent_tokenize(doc):
    sent_text = nltk.sent_tokenize(doc) # this gives you a list of sentences
    return sent_text

def word_tokenize(sent):
    tokenized_text = nltk.word_tokenize(sent)  # this gives you a list of words
    tokenized_text = [token.lower() for token in tokenized_text if token not in punctuation_list]  
    # optional: convert all words to lower case
    return tokenized_text

def readfile(filename):
    with open(filename,'r',encoding='UTF-8') as f:
        content = f.readlines()
    content = [x.strip() for x in content] 
    #strip()读出有效文件，形成一个list
    #split()读成有效文件，根据一行来形成一个list
    return content

#padding the sentence
#sentences是一个影评，就是一个train_data_word[0]
#max_words是影评中句子的最大含词量
#max_sents是影评中最大的句子个数
#保证每个影评的句子个数和句子长度都一样
def pad_sent(sentences, max_words, max_sents):
    """
    Pads sequences to the same length.
    Input: sentences - List of lists, where each element is a sequence.
    - max_words: Int, maximum length of all sequences.
    """
    # pad sentences in a doc
    sents_padded = pad_sequences(sentences, maxlen=max_words, padding='post') 
    # pad a doc to have equal number of sentences
    if len(sents_padded) < max_sents:
        doc_padding = np.zeros((max_sents-len(sents_padded),max_words), dtype = int)
        sents_padded = np.append(doc_padding, sents_padded, axis=0)
    else:
        sents_padded = sents_padded[:max_sents]
    return sents_padded

#build from word to integer as the input of ''
def build_vocab(corpus):
    """
    Builds a vocabulary mapping from word to index based on the corpus.
    Input: list of all samples in the training data
    Return: OrderedDict - vocabulary mapping from word to integer.
    """
    # Build vocabulary
    corpus_2d = []  # convert 3d corpus to 2d list
    for doc in corpus:
        for sent in doc:
            corpus_2d.append(sent)
    word_counts = Counter(itertools.chain(*corpus_2d))
    # Mapping from index to word (type: list)
    vocabulary = ['<PAD/>', '<UKN/>']   # 0 for padding, 1 for unknown words
    vocabulary = vocabulary + [x[0] for x in word_counts.most_common()]
    # Mapping from word to index
    #如何避免呢
    vocab2int = OrderedDict({x: i for i, x in enumerate(vocabulary)})
    return vocab2int

#****这个corpus是几维呢
def build_input_data(corpus, vocab2int, max_words, max_sents):
    """
    Maps words in the corpus to integers based on a vocabulary.
    Also pad the sentences and documents into fixed shape
    Input: corpus - list of samples, each sample is a list of sentences, each sentence is a list of words
    """
    corpus_int = [[[getWordIdx(word, vocab2int) for word in sentence]for sentence in sample] for sample in corpus]
    corpus_padded = []
    for doc in corpus_int:
        corpus_padded.append(pad_sent(doc, max_words, max_sents))
    corpus_padded = np.array(corpus_padded)    
    return corpus_padded

def load_embedding_matrix_gensim(embed_path, vocab2int, EMBEDDING_DIM):
    """
    load Word2Vec using gensim: 300x1 word vecs from Google (Mikolov) word2vec: GoogleNews-vectors-negative300.bin
    return embedding_matrix 
    embedding_matrix[i] is the embedding for 'vocab2int' integer index i
    """
    word2vec_model = gensim.models.KeyedVectors.load_word2vec_format(embed_path, binary=True)
    embeddings = {}
    embeddings['<PAD/>'] = np.zeros(EMBEDDING_DIM) # Zero vector for '<PAD/>' word
    embedding_UKN = np.random.uniform(-0.10, 0.10, EMBEDDING_DIM)  # Vector of small random numbers for unknown words
    # embedding_UKN = vector / np.linalg.norm(embedding_UKN)   # Normalize to unit vector
    embeddings['<UKN/>'] = embedding_UKN

    for word in word2vec_model.vocab:
        embeddings[word] = word2vec_model[word]

    embedding_matrix = np.zeros((len(vocab2int) , EMBEDDING_DIM))
    for word, i in vocab2int.items():
        embedding_vector = embeddings.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
        else:   # word is unknown
            embedding_vector = np.random.uniform(-0.10, 0.10, EMBEDDING_DIM)  # Vector of small random numbers for unknown words
            # embedding_vector = vector / np.linalg.norm(embedding_vector)   # Normalize to unit vector
            embedding_matrix[i] = embedding_vector

    return embedding_matrix


In [3]:
#**********loading data
#get the movie review "list of string"
path_pos = 'data/unprocessed/sorted_data/apparel/positive.review'
path_neg = 'data/unprocessed/sorted_data/apparel/negative.review'

#read the file
file_pos = open(path_pos,'r',encoding='windows-1252')
file_pos = file_pos.read()

file_neg = open(path_neg,'r',encoding='windows-1252')
file_neg = file_neg.read()

#extact the file
positive = BeautifulSoup(file_pos)
positive = positive.find_all('review_text')#get all of the positive reviews
for i in range(len(positive)):#convet the elements in postive to the string type
    positive[i] = str(positive[i])

negative = BeautifulSoup(file_neg)
negative = negative.find_all('review_text')#get all the positive reviews
for i in range(len(negative)):#convet the elements in negative to the string type
    negative[i] = str(negative[i])
    
#eliminate the <review_text></review_text>tag in the reviews and normalize the text
positive_doc = []
for review in positive:
    review = normalize_text(review[14:-15])
    review = review.lower()
    positive_doc.append(review)

negative_doc = []
for review in negative:
    review = normalize_text(review[14:-15])
    review = review.lower()
    negative_doc.append(review)
    
#merge the data
data = negative_doc + positive_doc

#get the train label
pos_label = [1 for i in range(len(positive_doc))]
neg_label = [0 for i in range(len(negative_doc))]
train_label = pos_label + neg_label

In [4]:
#loading data
file = open('pickle_New_Data/embedding_matrix.pickle','rb')
embedding_matrix = pickle.load(file)

file = open('pickle_New_Data/vocab_to_int.pickle','rb')
vocab_to_int = pickle.load(file)
file.close()

In [33]:
#get the maxlen word
maxlen_word = 0

list_maxlen_word = []
for i in range(len(data)):
    list_maxlen_word.append((len(data[i])))

#get the max words
list_maxlen_word1 = sorted(list_maxlen_word)
maxlen_word = list_maxlen_word[int(len(list_maxlen_word1)*0.95)] 

In [34]:
#map the word to integer
data2int = []
word2int = []
for review in data:
    word2int =[getWordIdx(word, vocab_to_int) for word in review]
    data2int.append(word2int)

#padding for data
sequence = keras.preprocessing.sequence
data_padded = sequence.pad_sequences(data2int, maxlen=maxlen_word)

In [35]:
#get the train label
pos_label = [1 for i in range(len(positive_doc))]
neg_label = [0 for i in range(len(negative_doc))]
label = pos_label + neg_label

In [36]:
#split the data to train and test dataset
train_data, test_data, train_label, test_label = train_test_split(
data_padded, label, test_size=0.2, random_state=42)

In [37]:
#the shape of the data
print('train data shape:',train_data.shape)
print('test data shape:',test_data.shape)
print('embedding_matrix shape:', embedding_matrix.shape)
#the size of vocabulary
vocab_size = len(vocab_to_int)
print('vocabulary size:', vocab_size)

train data shape: (1600, 123)
test data shape: (400, 123)
embedding_matrix shape: (8023, 300)
vocabulary size: 8023


In [42]:
# # hyper-parameters
# batch_size = 50
# epoch_num = 15
# drop_out = 0.2
# gru_dim = 50
# lstm_dim = 50
# num_filter = 100
# window_size = 3
# atten_dim = 50

# hyper-parameters hierarchical
gru_dim = 128
dropout_rate = 0.3
atten_dim = 50
lstm_dim = 50

batch_size = 50
epoch_num = 15

categorical_label = True

if categorical_label:
    train_label_cat = np_utils.to_categorical(train_label)
    test_label_cat = np_utils.to_categorical(test_label)
categorical_label = True

if categorical_label:
    train_label_cat = np_utils.to_categorical(train_label)
    test_label_cat = np_utils.to_categorical(test_label)

## CNN 

In [39]:
#build the model
input_data = Input(shape=(maxlen_word,))
embeded = Embedding(vocab_size, embedding_matrix.shape[1], input_length=maxlen_word, 
                            weights=[embedding_matrix], trainable=False)(input_data)
hidden_layer = Convolution1D(nb_filter=100,
                            filter_length=3,
                            border_mode='same',
                            activation='tanh',
                            subsample_length=1
                            )(embeded)
cnn_out = GlobalMaxPooling1D()(hidden_layer)
preds = Dense(2, activation='softmax')(cnn_out) # categorical output
model = Model(input_data, preds)
model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['acc'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 123)               0         
_________________________________________________________________
embedding_3 (Embedding)      (None, 123, 300)          2406900   
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 123, 100)          90100     
_________________________________________________________________
global_max_pooling1d_2 (Glob (None, 100)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 202       
Total params: 2,497,202
Trainable params: 90,302
Non-trainable params: 2,406,900
_________________________________________________________________


C:\Users\wyl19\Anaconda3\envs\tensorflow_cpu\lib\site-packages\ipykernel\__main__.py:9: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(activation="tanh", filters=100, kernel_size=3, strides=1, padding="same")`


In [40]:
print('Training...')
for i in range(epoch_num):
    print('Training for epoch {}/{}'.format(i+1,epoch_num))
    if categorical_label:
        model.fit(train_data, train_label_cat, batch_size=batch_size,epochs=1)
    else:
        model.fit(train_data, train_label, batch_size=batch_size,epochs=1)
        
    print('Evaluating...')
    pred_test_prob = model.predict(test_data, batch_size=batch_size, verbose=True)
    # predict the class label
    if pred_test_prob.shape[-1]>1:
        pred_test = pred_test_prob.argmax(axis=-1)
    else:
        pred_test = (pred_test_prob>0.5).astype('int32')
        pred_test = pred_test.reshape(pred_test.shape[0])

    acc = np.sum(pred_test == test_label) / float(len(test_label))

    print("Accuracy: %.4f" % (acc))   
    print(classification_report(test_label, pred_test, digits=4, labels=[0, 1]))

Training...
Training for epoch 1/15
Epoch 1/1
1600/1600 [==============================] - 2s 1ms/step - loss: 0.6730 - acc: 0.5706
Evaluating...
400/400 [==============================] - 0s 573us/step
Accuracy: 0.6800
              precision    recall  f1-score   support

           0     0.6911    0.6567    0.6735       201
           1     0.6699    0.7035    0.6863       199

   micro avg     0.6800    0.6800    0.6800       400
   macro avg     0.6805    0.6801    0.6799       400
weighted avg     0.6805    0.6800    0.6798       400

Training for epoch 2/15
Epoch 1/1
1600/1600 [==============================] - 2s 1ms/step - loss: 0.6210 - acc: 0.6725
Evaluating...
400/400 [==============================] - 0s 466us/step
Accuracy: 0.6925
              precision    recall  f1-score   support

           0     0.7708    0.5522    0.6435       201
           1     0.6484    0.8342    0.7297       199

   micro avg     0.6925    0.6925    0.6925       400
   macro avg     0.7096    

Training for epoch 15/15
Epoch 1/1
1600/1600 [==============================] - 2s 1ms/step - loss: 0.3862 - acc: 0.8562
Evaluating...
400/400 [==============================] - 0s 501us/step
Accuracy: 0.7500
              precision    recall  f1-score   support

           0     0.7349    0.7861    0.7596       201
           1     0.7676    0.7136    0.7396       199

   micro avg     0.7500    0.7500    0.7500       400
   macro avg     0.7512    0.7498    0.7496       400
weighted avg     0.7511    0.7500    0.7496       400

## LSTM

In [44]:
#build LSTM model
input_data = Input(shape=(maxlen_word,))
embeded = Embedding(vocab_size, embedding_matrix.shape[1], input_length=maxlen_word, 
                            weights=[embedding_matrix], trainable=False)(input_data)
hidden_layer = LSTM(lstm_dim, activation = 'relu',dropout=dropout_rate, recurrent_dropout=dropout_rate)(embeded)
Final = Dense(2,activation = 'softmax')(hidden_layer)

#compile the model
model = Model(inputs=[input_data], outputs=[Final])
model.compile(loss='binary_crossentropy', optimizer='Adam', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 123)               0         
_________________________________________________________________
embedding_6 (Embedding)      (None, 123, 300)          2406900   
_________________________________________________________________
lstm_1 (LSTM)                (None, 50)                70200     
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 102       
Total params: 2,477,202
Trainable params: 70,302
Non-trainable params: 2,406,900
_________________________________________________________________


In [45]:
print('Training...')
for i in range(epoch_num):
    print('Training for epoch {}/{}'.format(i+1,epoch_num))
    if categorical_label:
        model.fit(train_data, train_label_cat, batch_size=batch_size,epochs=1)
    else:
        model.fit(train_data, train_label, batch_size=batch_size,epochs=1)
        
    print('Evaluating...')
    pred_test_prob = model.predict(test_data, batch_size=batch_size, verbose=True)
    # predict the class label
    if pred_test_prob.shape[-1]>1:
        pred_test = pred_test_prob.argmax(axis=-1)
    else:
        pred_test = (pred_test_prob>0.5).astype('int32')
        pred_test = pred_test.reshape(pred_test.shape[0])

    acc = np.sum(pred_test == test_label) / float(len(test_label))

    print("Accuracy: %.4f" % (acc))   
    print(classification_report(test_label, pred_test, digits=4, labels=[0, 1]))

Training...
Training for epoch 1/15
Epoch 1/1
1600/1600 [==============================] - 7s 4ms/step - loss: 0.6932 - acc: 0.5369
Evaluating...
400/400 [==============================] - 1s 2ms/step
Accuracy: 0.5050
              precision    recall  f1-score   support

           0     0.8000    0.0199    0.0388       201
           1     0.5013    0.9950    0.6667       199

   micro avg     0.5050    0.5050    0.5050       400
   macro avg     0.6506    0.5074    0.3528       400
weighted avg     0.6514    0.5050    0.3512       400

Training for epoch 2/15
Epoch 1/1
1600/1600 [==============================] - 3s 2ms/step - loss: 0.6886 - acc: 0.5450
Evaluating...
400/400 [==============================] - 0s 736us/step
Accuracy: 0.5100
              precision    recall  f1-score   support

           0     0.5104    0.6119    0.5566       201
           1     0.5094    0.4070    0.4525       199

   micro avg     0.5100    0.5100    0.5100       400
   macro avg     0.5099    0.

Training for epoch 12/15
Epoch 1/1
1600/1600 [==============================] - 4s 2ms/step - loss: 0.6718 - acc: 0.5750
Evaluating...
400/400 [==============================] - 0s 855us/step
Accuracy: 0.5400
              precision    recall  f1-score   support

           0     0.5475    0.4876    0.5158       201
           1     0.5339    0.5930    0.5619       199

   micro avg     0.5400    0.5400    0.5400       400
   macro avg     0.5407    0.5403    0.5388       400
weighted avg     0.5407    0.5400    0.5387       400

## GRU 

In [46]:
#build GRU model
input_data = Input(shape=(maxlen_word,))
embeded = Embedding(vocab_size, embedding_matrix.shape[1], input_length=maxlen_word, 
                            weights=[embedding_matrix], trainable=False)(input_data)
hidden_layer = GRU(gru_dim, activation = 'relu',dropout=dropout_rate, recurrent_dropout=dropout_rate)(embeded)
Final = Dense(2,activation = 'softmax')(hidden_layer)

#compile the model
model = Model(inputs=[input_data], outputs=[Final])
model.compile(loss='binary_crossentropy', optimizer='Adam', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         (None, 123)               0         
_________________________________________________________________
embedding_7 (Embedding)      (None, 123, 300)          2406900   
_________________________________________________________________
gru_1 (GRU)                  (None, 128)               164736    
_________________________________________________________________
dense_4 (Dense)              (None, 2)                 258       
Total params: 2,571,894
Trainable params: 164,994
Non-trainable params: 2,406,900
_________________________________________________________________


In [47]:
print('Training...')
for i in range(epoch_num):
    print('Training for epoch {}/{}'.format(i+1,epoch_num))
    if categorical_label:
        model.fit(train_data, train_label_cat, batch_size=batch_size,epochs=1)
    else:
        model.fit(train_data, train_label, batch_size=batch_size,epochs=1)
        
    print('Evaluating...')
    pred_test_prob = model.predict(test_data, batch_size=batch_size, verbose=True)
    # predict the class label
    if pred_test_prob.shape[-1]>1:
        pred_test = pred_test_prob.argmax(axis=-1)
    else:
        pred_test = (pred_test_prob>0.5).astype('int32')
        pred_test = pred_test.reshape(pred_test.shape[0])

    acc = np.sum(pred_test == test_label) / float(len(test_label))

    print("Accuracy: %.4f" % (acc))   
    print(classification_report(test_label, pred_test, digits=4, labels=[0, 1]))

Training...
Training for epoch 1/15
Epoch 1/1
1600/1600 [==============================] - 7s 4ms/step - loss: 0.6926 - acc: 0.5250
Evaluating...
400/400 [==============================] - 4s 10ms/step
Accuracy: 0.5325
              precision    recall  f1-score   support

           0     0.5318    0.5821    0.5558       201
           1     0.5333    0.4824    0.5066       199

   micro avg     0.5325    0.5325    0.5325       400
   macro avg     0.5326    0.5323    0.5312       400
weighted avg     0.5326    0.5325    0.5313       400

Training for epoch 2/15
Epoch 1/1
1600/1600 [==============================] - 6s 4ms/step - loss: 0.6857 - acc: 0.5681
Evaluating...
400/400 [==============================] - 1s 1ms/step
Accuracy: 0.5200
              precision    recall  f1-score   support

           0     0.5398    0.3035    0.3885       201
           1     0.5122    0.7387    0.6049       199

   micro avg     0.5200    0.5200    0.5200       400
   macro avg     0.5260    0.5

Training for epoch 15/15
Epoch 1/1
1600/1600 [==============================] - 6s 4ms/step - loss: 0.6422 - acc: 0.6256
Evaluating...
400/400 [==============================] - 1s 1ms/step
Accuracy: 0.5775
              precision    recall  f1-score   support

           0     0.5909    0.5174    0.5517       201
           1     0.5670    0.6382    0.6005       199

   micro avg     0.5775    0.5775    0.5775       400
   macro avg     0.5789    0.5778    0.5761       400
weighted avg     0.5790    0.5775    0.5760       400

## biGRU 

In [48]:
class AttLayer(Layer):
    def __init__(self, attention_dim):
        self.init = initializers.get('normal')
        self.supports_masking = True
        self.attention_dim = attention_dim
        super(AttLayer, self).__init__()

    def build(self, input_shape):
        assert len(input_shape) == 3
        self.W = K.variable(self.init((input_shape[-1], self.attention_dim)))
        self.b = K.variable(self.init((self.attention_dim, )))
        self.u = K.variable(self.init((self.attention_dim, 1)))
        self.trainable_weights = [self.W, self.b, self.u]
        super(AttLayer, self).build(input_shape)

    def compute_mask(self, inputs, mask=None):
        return mask

    def call(self, x, mask=None):
        # size of x :[batch_size, sel_len, attention_dim]
        # size of u :[batch_size, attention_dim]
        # uit = tanh(xW+b)
        uit = K.tanh(K.bias_add(K.dot(x, self.W), self.b))
        ait = K.dot(uit, self.u)
        ait = K.squeeze(ait, -1)

        ait = K.exp(ait)

        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            ait *= K.cast(mask, K.floatx())
        ait /= K.cast(K.sum(ait, axis=1, keepdims=True) + K.epsilon(), K.floatx())
        ait = K.expand_dims(ait)
        weighted_input = x * ait
        output = K.sum(weighted_input, axis=1)

        return output

    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[-1])

In [49]:
#build GRU model
input_data = Input(shape=(maxlen_word,))
embeded = Embedding(vocab_size, embedding_matrix.shape[1], input_length=maxlen_word, 
                            weights=[embedding_matrix], trainable=False)(input_data)
hidden_layer = Bidirectional(GRU(gru_dim, activation = 'relu',dropout=dropout_rate, recurrent_dropout=dropout_rate,return_sequences = True))(embeded)
att_out = AttLayer(atten_dim)(hidden_layer)
Final = Dense(2,activation = 'softmax')(att_out)

#compile the model
model = Model(inputs=[input_data], outputs=[Final])
model.compile(loss='binary_crossentropy', optimizer='Adam', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         (None, 123)               0         
_________________________________________________________________
embedding_8 (Embedding)      (None, 123, 300)          2406900   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 123, 256)          329472    
_________________________________________________________________
att_layer_1 (AttLayer)       (None, 256)               12900     
_________________________________________________________________
dense_5 (Dense)              (None, 2)                 514       
Total params: 2,749,786
Trainable params: 342,886
Non-trainable params: 2,406,900
_________________________________________________________________


In [50]:
print('Training...')
for i in range(epoch_num):
    print('Training for epoch {}/{}'.format(i+1,epoch_num))
    if categorical_label:
        model.fit(train_data, train_label_cat, batch_size=batch_size,epochs=1)
    else:
        model.fit(train_data, train_label, batch_size=batch_size,epochs=1)
        
    print('Evaluating...')
    pred_test_prob = model.predict(test_data, batch_size=batch_size, verbose=True)
    # predict the class label
    if pred_test_prob.shape[-1]>1:
        pred_test = pred_test_prob.argmax(axis=-1)
    else:
        pred_test = (pred_test_prob>0.5).astype('int32')
        pred_test = pred_test.reshape(pred_test.shape[0])

    acc = np.sum(pred_test == test_label) / float(len(test_label))

    print("Accuracy: %.4f" % (acc))   
    print(classification_report(test_label, pred_test, digits=4, labels=[0, 1]))

Training...
Training for epoch 1/15
Epoch 1/1
1600/1600 [==============================] - 29s 18ms/step - loss: 0.6953 - acc: 0.4988
Evaluating...
400/400 [==============================] - 3s 8ms/step
Accuracy: 0.5150
              precision    recall  f1-score   support

           0     0.5089    1.0000    0.6745       201
           1     1.0000    0.0251    0.0490       199

   micro avg     0.5150    0.5150    0.5150       400
   macro avg     0.7544    0.5126    0.3618       400
weighted avg     0.7532    0.5150    0.3633       400

Training for epoch 2/15
Epoch 1/1
1600/1600 [==============================] - 14s 8ms/step - loss: 0.6935 - acc: 0.5156
Evaluating...
400/400 [==============================] - 1s 3ms/step
Accuracy: 0.5750
              precision    recall  f1-score   support

           0     0.5552    0.7761    0.6473       201
           1     0.6218    0.3719    0.4654       199

   micro avg     0.5750    0.5750    0.5750       400
   macro avg     0.5885    0

Training for epoch 15/15
Epoch 1/1
1600/1600 [==============================] - 14s 9ms/step - loss: 0.5283 - acc: 0.7281
Evaluating...
400/400 [==============================] - 1s 3ms/step
Accuracy: 0.6925
              precision    recall  f1-score   support

           0     0.6711    0.7612    0.7133       201
           1     0.7209    0.6231    0.6685       199

   micro avg     0.6925    0.6925    0.6925       400
   macro avg     0.6960    0.6922    0.6909       400
weighted avg     0.6959    0.6925    0.6910       400